In [1]:
from efficientnet_pytorch import EfficientNet
import os
import pandas as pd
import torch
import cv2
import neptune.new as neptune


run = neptune.init(
    project="mod-cls-effnet", api_token="ANON"
)

https://app.neptune.ai/mrtahion/mod-cls-effnet/e/MOD-14
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


###  Создание набора аугментаций

In [2]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

def train_transform(img_size=240):
    transform = A.Compose([
        A.Flip(p=.25),
        A.ShiftScaleRotate(
                shift_limit=0.0625,
                scale_limit=0.1,
                rotate_limit=45,
                p=.25,
                interpolation=3,
            ),

        A.OneOf([
                    A.GaussNoise(p=0.5),
                    A.MultiplicativeNoise(p=0.5),
                ],
                p=0.1,
            ),
        A.OneOf(
                [
                    A.MotionBlur(
                        p=0.5,
                    ),
                    A.MedianBlur(
                        p=0.5,
                    ),
                    A.GaussianBlur(
                        p=0.5,
                    ),
                    A.GlassBlur(
                        p=0.5,
                    ),
                ],
                p=0.1,
            ),
        A.OneOf(
            [
                A.Sharpen(p=0.5),
                A.RandomBrightnessContrast(p=0.5),
            ],
            p=0.1,
        ),
        A.OneOf(
            [
                A.ChannelShuffle(p=0.1),
                A.HueSaturationValue(p=0.1),
                A.RGBShift(p=0.2)
            ],
            p=0.2
        ),
        A.InvertImg(p=0.1),
        A.LongestMaxSize(max_size=img_size, always_apply=True, interpolation=3),
        A.PadIfNeeded(min_height=img_size, min_width=img_size),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2()
    ])
    
    return transform
    
def test_transform(img_size=240):
    transform = A.Compose([
        A.LongestMaxSize(max_size=img_size, always_apply=True, interpolation=3),
        A.PadIfNeeded(min_height=img_size, min_width=img_size),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2()
    ])
    
    return transform

### Создание датасета

In [4]:
from torch.utils.data import Dataset, DataLoader

class ModDataset(Dataset):
    def __init__(self, csv_path, img_dir, transform=test_transform()):
        # Read the csv file
        self.data_info = pd.read_csv(csv_path)
        self.image_arr = np.asarray(self.data_info["filename"])
        self.label_arr = np.asarray(self.data_info["class"])
        
        self.transform=transform
        
        self.img_dir = img_dir
    
    def loader(self, img_path):
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        return img

    def __getitem__(self, index):
        # Get image name from the pandas df
        single_image_name = self.image_arr[index]
        # Open image
        img_as_img = self.loader(os.path.join(self.img_dir,single_image_name))
        if self.transform is not None:
            image_tranformed = self.transform(image=img_as_img)["image"]
            
        # Get label(class) of the image based on the cropped pandas column
        single_image_label = self.label_arr[index]

        return (image_tranformed, single_image_label)

    def __len__(self):
        return len(self.data_info.index)

### Инициализация модели и основных гиперпараметров

In [5]:
import torch.nn as nn


params ={
    "lr": 1e-4,
    "batch_size": 64,
    "device": "cuda:1",
    "num_classes": 30, 
    "num_workers": 8,
    "n_epoch": 10, 
    "amsgrad": True,
    "gamma": 0.9
    }

model = EfficientNet.from_pretrained('efficientnet-b1', num_classes=params["num_classes"]).to(params["device"])
optimizer = torch.optim.AdamW(model.parameters(), lr=params["lr"], amsgrad=params["amsgrad"])
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=params["gamma"])
criterion = nn.CrossEntropyLoss().to(params["device"])

run["config/model"] = type(model).__name__
run["config/criterion"] = type(criterion).__name__
run["config/optimizer"] = type(optimizer).__name__
run["config/scheduler"] = type(scheduler).__name__
run["config/params"] = params

Loaded pretrained weights for efficientnet-b1


### Создадим dataloader'ы

In [6]:
train_dataset = ModDataset(csv_path="PATH", img_dir="PATH", transform=train_transform())
test_dataset = ModDataset(csv_path="PATH", img_dir="PATH", transform=test_transform())

train_loader = DataLoader(
    train_dataset, batch_size=params["batch_size"], shuffle=True, num_workers=params["num_workers"], pin_memory=True, drop_last=True
)
test_loader = DataLoader(
    test_dataset, batch_size=params["batch_size"], shuffle=False, num_workers=params["num_workers"], pin_memory=True,
)


### Функции для тестирования и обучения

In [7]:
    
def train(train_loader, model, criterion, optimizer, params):
    model.train()
    
    for i, (images, target) in enumerate(train_loader, start=1):
        images = images.cuda(params["device"], non_blocking=True)
        target = target.cuda(params["device"], non_blocking=True)
        
        output = model(images)
        _, preds = torch.max(output, 1)
        loss = criterion(output, target)
        acc = (torch.sum(preds == target.data)) / len(target)
        
        run["train/batch/loss"].log(loss)
        run["train/batch/acc"].log(acc)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
@torch.no_grad()
def test(test_loader, model, criterion, params):
    model.eval()
    test_predictions = []
    test_labels = []
    
    for i, (images, target) in enumerate(test_loader, start=1):
        images = images.cuda(params["device"], non_blocking=True)
        target = target.data.cpu().numpy()
            
        output = model(images)
        _, preds = torch.max(output, 1)
        test_predictions.append(preds.data.cpu().numpy())
        test_labels.append(target)
    
    test_predictions =  np.concatenate(test_predictions)
    test_labels = np.concatenate(test_labels)
    acc = (test_predictions == test_labels).sum() / len(test_labels)
    
    run["test/acc"] = acc

### Запуск обучения

In [8]:
for epoch in range(1, params["n_epoch"] + 1):
    run["train/epoch/lr"].log(scheduler.get_last_lr())
    train(train_loader, model, criterion, optimizer, params)
    scheduler.step()

test(test_loader, model, criterion, params)

[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warn

### Сохраним результат

In [9]:
torch.save(model.state_dict(), "PATH")
run["io_files/model_checkpoint"].upload("PATH")


In [10]:
run.stop()

Shutting down background jobs, please wait a moment...
Done!


Waiting for the remaining 107 operations to synchronize with Neptune. Do not kill this process.


All 107 operations synced, thanks for waiting!
